In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from scipy.stats import randint as randint
from scipy.stats import uniform
from sklearn.model_selection import cross_val_score
import re # Регулярные выражения.
import string
import datetime
import pymystem3

try:
    from sklearn.model_selection import GridSearchCV
    from sklearn.model_selection import RandomizedSearchCV
    from sklearn.model_selection import StratifiedKFold
except ImportError:
    from sklearn.cross_validation import GridSearchCV
    from sklearn.cross_validation import RandomizedSearchCV
    from sklearn.cross_validation import StratifiedKFold


ohe = OneHotEncoder(sparse = False)
mystem=pymystem3.Mystem()
%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12,5)

Для вас представлена задача, сделанная на открытых данных.
Вам нужно предсказать цену товара по другим имеющимся характеристикам.
Метрика качества RMSLE.<br>
ЦЕНА в КОПЕЙКАХ!!
Но предсказания нужны в РУБЛЯХ
потому - сразу поделите в train прайс на 100

In [2]:
df_train = pd.read_pickle('./data/train_sample.pckl')

In [ ]:
df_train[:100]

In [ ]:
df_train.info()

In [12]:
def rub(price):
    price = price / 100
    return price

def name_extract (S):
   S = str(S)
   delim = "': '"
   delim2 = "', '"
   return re.split(delim2, re.split(delim, S)[1])[0].strip()

def subcategory_extract (S):
   S = str(S)
   delim = "'name': '"
   delim2 = "', '"
   return re.split(delim2, re.split(delim, S)[2])[0].strip()

def replace_logical(s):
   if s:
       return 1
   else:
       return 0
    
def transformYear(val):
   #year = datetime.datetime.fromtimestamp(int(df_train.date_created[0])).year
   year = datetime.datetime.fromtimestamp(int(val)).year
   return year

def transformMonth(val):
   month = datetime.datetime.fromtimestamp(int(val)).month
   return month

def get_latitude(val):
   res = val['latitude']
   return res

def get_longitude(val):
   res = val['longitude']
   return res

def mystem_func(text):
    return ''.join(mystem.lemmatize(text))

def data_preproc(df_input):
    df_output = df_input.copy() # копируем датафрейм
    df_output = df_output.drop(['category', 'description', 'id', 'images', 'subway', 'subcategory', 'can_buy', 'can_promote'], axis=1) #удаляем ненужные признаки
    df_output.price = df_train['price'].map(rub) # нормируем цену до рублей.
    df_output['category_name'] = df_output['fields'].map(name_extract)
    df_output['subcategory_name'] = df_output['fields'].map(subcategory_extract)
    df_output = df_output.drop(['fields'], axis=1)
    df_output.contacts_visible = df_output.contacts_visible.map(replace_logical)
    df_output.delivery_available = df_output.delivery_available.map(replace_logical)
    df_output.mortgage_available = df_output.mortgage_available.map(replace_logical)
    df_output.payment_available = df_output.payment_available.map(replace_logical)
    df_output['month'] = df_output['date_created'].map(transformMonth)
    df_output['year']=df_output['date_created'].map(transformYear)
    df_output = df_output.drop(['date_created'], axis=1)
    df_output['price'] = df_output['price'].astype(int)
    df_output = df_output[df_output['price']>50]
    df_output['latitude'] = df_output['location'].map(get_latitude)
    df_output['longitude'] = df_output['location'].map(get_longitude)
    df_output = df_output.drop(['location'], axis=1)
    df_output['name'] = df_output['name'].map(mystem_func)
    df_output = df_output[['contacts_visible', 'delivery_available', 'mortgage_available', 'name', 'payment_available', 'category_name', 'subcategory_name', 'month', 'year', 'latitude', 'longitude', 'price']]
    return df_output
    
    

In [13]:
df_train_preproc = df_train.pipe(data_preproc)

In [14]:
df_train_preproc[:100]

,contacts_visible,delivery_available,mortgage_available,name,payment_available,category_name,subcategory_name,month,year,latitude,longitude,price
0,1,0,0,сумка DG\n,1,Женский гардероб,Аксессуары,4,2017,55.806888,37.546077,1999
1,1,0,0,комплект\n,0,Детский гардероб,Аксессуары,10,2016,55.692979,37.872337,350
2,1,0,0,пальтишко демисезонный\n,0,Детский гардероб,Верхняя одежда,9,2016,55.639011,37.349378,300
3,1,0,0,Attipas\n,1,Детский гардероб,Обувь,10,2016,55.847334,37.495834,800
5,1,0,0,жилет теплый\n,1,Детский гардероб,Верхняя одежда,8,2017,60.044826,30.355460,500
6,1,0,0,"босоножка, сабо\n",0,Женский гардероб,Обувь,6,2017,56.367960,43.818292,300
7,1,0,0,жесткий диск для ноутбук Seagate 320gb\n,1,Компьютерная техника,Комплектующие и запчасти,3,2017,51.638327,39.267098,1500
8,1,0,0,ковер 324 на 240\n,1,Для дома и дачи,Текстиль и ковры,4,2017,45.017036,41.923240,2000
10,1,0,0,кенгуру\n,0,Детские товары,Другое,12,2016,53.279983,50.273564,350
11,1,0,0,машинка автомат\n,1,Бытовая техника,Стиральные машины,11,2016,53.727380,84.926507,3500


In [11]:
model = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1,1),
                              max_df=0.835)),
    ('clf', LogisticRegression(random_state=13)),
])
cv = StratifiedKFold(n_splits=5, random_state=13, shuffle=True)

In [ ]:
X = df_train_preproc.loc[:, 'information'].values
y = df_preproc.loc[:, ].values